In [1]:
!apt install tesseract-ocr
!apt install libtesseract-dev
!pip install pytesseract

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import json
import requests
from PIL import Image
import pytesseract
import numpy as np
import requests
from io import BytesIO

# Dataset

To obtain the dataset we have done web scrapping in the https://www.org/ web. First we obtain the number of poems of a given period and then we iterate over all the pages to obtain the title, author and the poem itself.

In [3]:
def extract_total_results(url):
    response = requests.get(url)
    data = response.json()
    total_results = data['TotalResults']
    return total_results

In [4]:
def extract_text_from_image(url):
  try:
        response = requests.get(url)
        img_bytes = BytesIO(response.content)
        img1 = np.array(Image.open(img_bytes))
        text = pytesseract.image_to_string(img1)
        text = text.replace('\n', ' ')
        return text
  except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [ ]:
url = 'https://www' # Add Web Page URL

response = requests.get(url)
poem_soup=BeautifulSoup(response.text, 'html.parser')
poem=poem_soup.find(class_='c-assetStack-media')
img = poem.find('img')
src_value = img['src']
print(src_value)
text=extract_text_from_image(src_value)

print(text)

In [6]:
def extract_poems(url,period):
    response = requests.get(url)
    data = response.json()

    # Extract attributes from each entry
    entries = []
    for entry in data['Entries']:
        entry_data = {
            'id': entry['id'],
            'title': entry['title'],
            'author': entry['author'],
            'snippet': entry['snippet'],
            'link': entry['link'],
            'categories': [category['title'] for category in entry['categories']],
            'period': period
        }
        
        # Fetch the poem content from the link
        poem_response = requests.get(entry_data['link'])
        poem_soup = BeautifulSoup(poem_response.text, 'html.parser')
        poem = poem_soup.find(class_='o-poem')
        if poem != None:
          try:
            poem_text = poem.get_text(separator=' ', strip=True) # If we wanted to take into accouent the form of the poem use \n as separator
          except Exception as e:
            print(f"An error occurred: {e}")
            poem_text = None
          # Add poem text to the entry data
          entry_data['poem'] = poem_text
        else: # When the poem is an image
          poem=poem_soup.find(class_='c-assetStack-media')
          if poem != None: # If there is a class c-assetStack-media
            img = poem.find('img')
            if img == None: # If the image is not found
              entry_data['poem'] = None
            else: # If the image is found
              src_value = img['src']
              poem_text = extract_text_from_image(src_value)
              entry_data['poem'] = poem_text
          else: # If there is no class c-assetStack-media, this means that the poem is an audio or other data types
            entry_data['poem'] = None

        entries.append(entry_data)

    # Create a pandas DataFrame
    df = pd.DataFrame(entries)
    return df

In [7]:
# Initialize an empty list to store individual DataFrames
dfs = []
period_ids = {
    'Middle English': 158,
    'Augustan': 149,
    'Renaissance': 163,
    'Romantic': 164,
    'Victorian': 165,
    'Fugitive': 153,
    'Georgian': 154,
    'Harlem Renaissance': 155,
    'Imagist': 156,
    'Modern': 159,
    'Objectivist': 162,
    'Beat': 150,
    'Black Arts Movement': 304,
    'Black Mountain': 151,
    'Confessional': 152,
    'Language Poetry': 157,
    'New York School': 160,
    'New York School (2nd Generation)': 161
}

In [8]:
# #Loop over all the periods to find how many entries they have
for i in period_ids:
  total_results = extract_total_results("https://www.sort_by=title&school-period={}".format(period_ids[i]))
  print("Extracting poems of ", i)
  # Loop through the URLs
  for j in range(total_results // 20 + 1): # 20 poems per page
      url = "https://www.page={}&sort_by=title&school-period={}".format(j+1, period_ids[i])
      # Extract DataFrame from each URL and append to the list
      dfs.append(extract_poems(url,i))



In [9]:
# # Concatenate all DataFrames into a single DataFrame
final_df = pd.concat(dfs, ignore_index=True)

# Display the final DataFrame
print(len(final_df))
final_df.head()

final_df.to_csv("poems_data.csv", index=False, quoting=1)

### Full Poems Extraction for good Tokenization

In [10]:
def extract_poems(url):
    response = requests.get(url)
    data = response.json()

    # Extract attributes from each entry
    entries = []
    for entry in data['Entries']:
        entry_data = {
            'id': entry['id'],
            'title': entry['title'],
            'author': entry['author'],
            'snippet': entry['snippet'],
            'link': entry['link'],
            'categories': [category['title'] for category in entry['categories']]            
        }

        # Fetch the poem content from the link
        poem_response = requests.get(entry_data['link'])
        poem_soup = BeautifulSoup(poem_response.text, 'html.parser')
        poem = poem_soup.find(class_='o-poem')
        if poem != None:
          try:
            poem_text = poem.get_text(separator=' ', strip=True) # If we wanted to take into accouent the form of the poem use \n as separator
          except Exception as e:
            print(f"An error occurred: {e}")
            poem_text = None
          # Add poem text to the entry data
          entry_data['poem'] = poem_text
        else: # When the poem is an image
          poem=poem_soup.find(class_='c-assetStack-media')
          if poem != None: # If there is a class c-assetStack-media
            img = poem.find('img')
            if img == None: # If the image is not found
              entry_data['poem'] = None
            else: # If the image is found
              src_value = img['src']
              poem_text = extract_text_from_image(src_value)
              entry_data['poem'] = poem_text
          else: # If there is no class c-assetStack-media, this means that the poem is an audio or other data types
            entry_data['poem'] = None

        entries.append(entry_data)

    # Create a pandas DataFrame
    df = pd.DataFrame(entries)
    return df




In [11]:
import time
total_results = extract_total_results("https://www.org/ajax/poems?page=1")
dfs = []
n_poems = 0
count =0

for j in range(2101,total_results // 20 + 1): # 20 poems per page
    start_time = time.time()

    url = "https://www.org/ajax/poems?page={}".format(j+1)
    # Extract DataFrame from each URL and append to the list
    dfs.append(extract_poems(url))

    n_poems += 20

    count += 1
    if count == 5:
        print("We have counted already ", n_poems, ",", (time.time() - start_time), "seconds have passed.")
        count = 0
        start_time = 0

We have counted already  100 , 32.248294830322266 seconds have passed.
We have counted already  200 , 26.4195818901062 seconds have passed.
We have counted already  300 , 31.873921632766724 seconds have passed.
We have counted already  400 , 56.13200354576111 seconds have passed.
We have counted already  500 , 58.59949707984924 seconds have passed.
We have counted already  600 , 57.18841743469238 seconds have passed.
We have counted already  700 , 57.89660143852234 seconds have passed.
We have counted already  800 , 64.29464888572693 seconds have passed.
We have counted already  900 , 58.9615204334259 seconds have passed.
We have counted already  1000 , 62.31735610961914 seconds have passed.
We have counted already  1100 , 58.395796060562134 seconds have passed.
We have counted already  1200 , 59.80884552001953 seconds have passed.
We have counted already  1300 , 57.53302621841431 seconds have passed.
We have counted already  1400 , 58.98824191093445 seconds have passed.
We have counte

In [12]:
# Concatenate all DataFrames into a single DataFrame
df = pd.concat(dfs, ignore_index=True)

# Display the final DataFrame
print(len(df))
df.head()

df.to_csv("full_poems_data.csv", index=False, quoting=1)

5451


<div style="background-color:green; padding:10px">
Notice that we have been iterating through this notebook as it took a lot of time to process everything. We have been merging datasets in the file named "Merging_Datasets"
</div>
